MODELO DE PREDICCIÓN DE SENTIMIENTOS

Pasos para crear un modelo de sentimientos:

1.- Entendiendo el análisis de sentimientos: El análisis de sentimientos es un área de la inteligencia artificial que se centra en la identificación y extracción de opiniones, emociones o actitudes en textos. Por ejemplo, se puede utilizar para determinar si las opiniones de los clientes sobre un producto son positivas, negativas o neutrales. Esta técnica es especialmente útil para empresas que buscan obtener información valiosa sobre sus productos o servicios, así como para investigadores y científicos de datos que buscan analizar grandes volúmenes de texto.

2.- Preparación de los datos: Para entrenar y evaluar un modelo de clasificación de texto, necesitamos un conjunto de datos etiquetados. Esto implica recopilar una serie de textos y asignarles una etiqueta que indique su sentimiento (positivo, negativo o neutral). A menudo, este proceso puede ser costoso y requiere tiempo, pero existen recursos disponibles en línea, como el conjunto de datos de opiniones de películas de IMDb, que podemos utilizar para este propósito.

3.- Preprocesamiento de los datos: Antes de alimentar nuestros datos a un modelo de machine learning, es necesario realizar ciertos pasos de preprocesamiento, como:

- Convertir todos los textos a minúsculas
- Eliminar signos de puntuación y caracteres especiales
- Tokenizar el texto en palabras individuales
- Eliminar palabras vacías (stop words)
- Aplicar lematización o derivación (stemming) para reducir las palabras a su raíz

4.- Extracción de características: Una vez que hayamos preprocesado nuestros datos, necesitamos convertirlos en un formato que los modelos de machine learning puedan entender. Para ello, podemos utilizar técnicas como la bolsa de palabras (Bag of Words) o la ponderación de frecuencia inversa de documentos (TF-IDF).

5.- Creación y entrenamiento del modelo: Con nuestros datos preprocesados y las características extraídas, podemos crear y entrenar un modelo de clasificación de texto utilizando scikit-learn. Algunos de los algoritmos más comunes para este propósito incluyen la regresión logística, el clasificador Naive Bayes y las máquinas de vectores de soporte (SVM).

6.- Evaluación del modelo: Para evaluar el rendimiento de nuestro modelo, podemos utilizar métricas como la precisión, la exhaustividad (recall) y la puntuación F1. También podemos utilizar técnicas de validación cruzada para obtener una estimación más precisa del rendimiento del modelo en datos no vistos.

7.- Optimización del modelo: Si el rendimiento de nuestro modelo no es satisfactorio, podemos intentar optimizarlo mediante la selección de características, la ajuste de hiperparámetros o la experimentación con diferentes algoritmos de clasificación.

8.- Implementación del modelo: Una vez que hayamos creado y optimizado nuestro modelo de análisis de sentimientos, podemos implementarlo en aplicaciones en tiempo real, como monitoreo de redes sociales, análisis de comentarios de clientes o clasificación de correos electrónicos según su tono emocional.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
#MODELO QUE DEVUELVE LAS METRICAS DE ACCURACY, PRECISION Y RECALL

# Descargar recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Leyendo y guardando los datos en un DataFrame
positivo = pd.read_csv("data/books/positive.review", names=["texto"])
negativo = pd.read_csv("data/books/negative.review", names=["texto"])

# Inicializar el stemmer y obtener la lista de stopwords
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Función para procesar el texto
def process_text(text):
    # Tokenización
    tokens = word_tokenize(text)
    # Convertir a minúsculas y eliminar stopwords, caracteres especiales y de puntuación
    filtered_tokens = [ps.stem(token.lower()) for token in tokens if token.isalnum() and len(token) >= 3 and token.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Aplicar la función a cada texto en positivo y negativo
positivo['processed_text'] = positivo['texto'].apply(process_text)
negativo['processed_text'] = negativo['texto'].apply(process_text)

# Crear una columna para la etiqueta
positivo['sentiment'] = 1
negativo['sentiment'] = 0

# Combinar los datos procesados en un solo DataFrame
all_data = pd.concat([positivo[['processed_text', 'sentiment']], negativo[['processed_text', 'sentiment']]], ignore_index=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(all_data['processed_text'], all_data['sentiment'], test_size=0.2, random_state=42)

# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar y transformar los datos de entrenamiento
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transformar los datos de prueba
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Inicializar y entrenar el clasificador Naive Bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Predecir en los datos de prueba
y_pred = nb_classifier.predict(X_test_tfidf)

# Calcular métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Imprimir métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)